# Cox生存分析

* `mydir`：自己的数据
* `ostime_column`: 数据对应的生存时间，不一定非的是OST，也可以是DST、FST等。
* `os`：生存状态，不一定非的是OS，也可以是DS、FS等。

In [ ]:
from lifelines import CoxPHFitter
import pandas as pd
from onekey_algo.custom.components.comp1 import normalize_df
from sklearn.model_selection import train_test_split
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.comp1 import fillna

event_col = get_param_in_cwd('event_col', 'label')
duration_col=get_param_in_cwd('duration_col', 'time')

def get_prediction(mn):
    prediction = pd.concat([pd.read_csv(f'results/{mn}_cox_predictions_{subset}.csv') for subset in get_param_in_cwd('subsets')],
                           axis=0)
    prediction.columns = ['ID', f'{mn}_Exp', mn]
    return prediction[['ID', f'{mn}']]

data = None
for mn in get_param_in_cwd('compare_models'):
    pred = get_prediction(mn)
    if data is None:
        data = pred
    else:
        data = pd.merge(data, pred)

# data = normalize_df(data, not_norm=['ID', 'group'])
label_data = pd.read_csv(get_param_in_cwd('survival_file'), dtype={'ID':str})
# label_data = fillna(label_data, fill_mod='50%')
data = pd.merge(data, label_data, on='ID', how='inner')
data = pd.merge(data, pd.read_csv('joinit_group.csv')[['ID', 'group']])
data

In [ ]:
from lifelines import CoxPHFitter
from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter
from lifelines.plotting import add_at_risk_counts
from lifelines.utils import concordance_index

metrics = []
for task in ['PFS', 'OS']:
    metric = [task]
    for subset in get_param_in_cwd('subsets'):
        subdata = data[data['group'] == subset]
        metric.append(concordance_index(subdata[f"{task}_time"], subdata[f"Full{task}"], subdata[task]))
    metrics.append(metric)
pd.DataFrame(metrics, columns=['Survival'] + get_param_in_cwd('subsets'))